In [1]:
import logging
import warnings
from pprint import pprint

import numpy as np
from openeye import oechem
from openff.qcsubmit.factories import OptimizationDatasetFactory
from openff.qcsubmit.workflow_components import StandardConformerGenerator
from openforcefield.topology import Molecule
from tqdm import tqdm

In [2]:
# Warnings that tell us we have undefined stereo and charged molecules
logging.getLogger("openforcefield").setLevel(logging.ERROR)
warnings.simplefilter("ignore")

# Dataset Preperation

Load in the SMILES patterns of the molecules to include:

In [3]:
with open("dataset.smi") as file:
    smiles_patterns = file.read().split("\n")
    smiles_patterns = [pattern for pattern in smiles_patterns if len(pattern) > 0]

Load in the molecules to be optimized:

In [4]:
molecules = []

for smiles_pattern in tqdm(smiles_patterns):

    molecule: Molecule = Molecule.from_smiles(smiles_pattern)

    try:
        molecule.generate_conformers(n_conformers=1)
    except Exception:
        print(f"Skipping {smiles_pattern} - OMEGA error.")
        continue

    molecules.append(molecule)

100%|██████████| 51/51 [00:07<00:00,  6.82it/s]


Skipping c1cc(ccc1N)[N+] - OMEGA error.


Prepare the main dataset from the molecule list.

In [5]:
dataset_factory = OptimizationDatasetFactory()

dataset_factory.add_workflow_component(
    StandardConformerGenerator(max_conformers=10, rms_cutoff=0.1, clear_existing=True)
)

dataset = dataset_factory.create_dataset(
    dataset_name="OpenFF Aniline Para Opt v1.0",
    tagline="Optimizations of diverse, para-substituted aniline derivatives.",
    description="This dataset contains a set of aniline derivatives which are para-substituted with "
    "groups of varying electron donating and withdrawing properties. This dataset was curated in "
    "an effort to improve and understand which functional groups will allow us to best interpolate "
    "between a planer and a pyramidal nitrogen.",
    molecules=molecules,
)

dataset.metadata.submitter = "simonboothroyd"
dataset.metadata.long_description_url = (
    "https://github.com/openforcefield/qca-dataset-submission/tree/master/"
    "submissions/"
    "2021-04-02-OpenFF-Aniline-Para-Opt-v1.0"
)

dataset.provenance["constructure"] = "0.0.1"

Preparation                   : 100%|███████████| 50/50 [00:01<00:00, 38.72it/s]


Make sure the molecules in the dataset match the input molecules

In [6]:
old_smiles = {Molecule.from_smiles(smiles).to_smiles(isomeric=False) for smiles in smiles_patterns}
new_smiles = {molecule.to_smiles(isomeric=False) for molecule in dataset.molecules}

assert len(old_smiles.symmetric_difference(new_smiles)) == 1

Describe the molecule in the dataset

In [7]:
confs = np.array([len(mol.conformers) for mol in dataset.molecules])

print("Number of unique molecules       ", dataset.n_molecules)
print("Number of filtered molecules     ", dataset.n_filtered)
print("Number of conformers             ", dataset.n_records)
print("Number of conformers min mean max",
      confs.min(), "{:6.2f}".format(confs.mean()), confs.max())

masses = []
for molecule in dataset.molecules:
    oemol = molecule.to_openeye()
    mass = oechem.OECalculateMolecularWeight(oemol)
    masses.append(mass)

print(f'Mean molecular weight: {np.mean(np.array(masses)):.2f}')
print(f'Max molecular weight: {np.max(np.array(masses)):.2f}')
print("Charges:", sorted(set(m.total_charge/m.total_charge.unit for m in dataset.molecules)))

Number of unique molecules        50
Number of filtered molecules      0
Number of conformers              223
Number of conformers min mean max 1   4.46 10
Mean molecular weight: 150.42
Max molecular weight: 343.84
Charges: [-1.0, 0.0, 1.0]


Describe the dataset

In [8]:
pprint(dataset.metadata.dict())

{'collection_type': 'OptimizationDataset',
 'creation_date': datetime.date(2021, 4, 2),
 'dataset_name': 'OpenFF Aniline Para Opt v1.0',
 'elements': {'Br', 'C', 'O', 'N', 'S', 'H', 'Cl', 'F'},
 'long_description': 'This dataset contains a set of aniline derivatives which '
                     'are para-substituted with groups of varying electron '
                     'donating and withdrawing properties. This dataset was '
                     'curated in an effort to improve and understand which '
                     'functional groups will allow us to best interpolate '
                     'between a planer and a pyramidal nitrogen.',
 'long_description_url': HttpUrl('https://github.com/openforcefield/qca-dataset-submission/tree/master/submissions/2021-04-02-OpenFF-Aniline-Para-Opt-v1.0', scheme='https', host='github.com', tld='com', host_type='domain', path='/openforcefield/qca-dataset-submission/tree/master/submissions/2021-04-02-OpenFF-Aniline-Para-Opt-v1.0'),
 'short_descrip

In [9]:
for spec, obj in dataset.qc_specifications.items():
    print("Spec:", spec)
    pprint(obj.dict())

Spec: default
{'basis': 'DZVP',
 'implicit_solvent': None,
 'method': 'B3LYP-D3BJ',
 'program': 'psi4',
 'spec_description': 'Standard OpenFF optimization quantum chemistry '
                     'specification.',
 'spec_name': 'default',
 'store_wavefunction': 'none'}


In [10]:
pprint(dataset.scf_properties)

[<SCFProperties.Dipole: 'dipole'>,
 <SCFProperties.Quadrupole: 'quadrupole'>,
 <SCFProperties.WibergLowdinIndices: 'wiberg_lowdin_indices'>,
 <SCFProperties.MayerIndices: 'mayer_indices'>]


Export the dataset.

In [11]:
dataset.export_dataset("dataset.json.bz2")
dataset.molecules_to_file("dataset.smi", "smi")

dataset.visualize("dataset.pdf", columns=8)